In [ ]:
%matplotlib inline

https://indico.cern.ch/event/628669/contributions/2600995/attachments/1463252/2266733/2017_05_26_Hmm_preApproval.pdf

In [ ]:
import uproot, awkward
import numpy as np
import sys
sys.path.append("/nfshome/jpata/gpu-analysis/")
if "/nfshome/jpata/.local/lib/python3.6/site-packages" in sys.path:
    sys.path.pop(sys.path.index("/nfshome/jpata/.local/lib/python3.6/site-packages"))
print(sys.path)
import hepaccelerate as ha
import os
os.environ["NUMBAPRO_NVVM"] = "/usr/local/cuda/nvvm/lib64/libnvvm.so"
os.environ["NUMBAPRO_LIBDEVICE"] = "/usr/local/cuda/nvvm/libdevice/"
#import cupy
print(os.environ["PYTHONPATH"])

In [ ]:
awkward.__version__, cupy.__version__

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#[jpata@login-1 ~]$ rsync -r -R --progress /mnt/hadoop/store/./data/Run2017*/SingleMuon/NANOAOD/Nano14Dec2018-v1 titans.hep.caltech.edu:/nvmedata/store/
files = OrderedDict()
files["dy"] = glob.glob("/nvmedata/store/mc/RunIIFall17NanoAOD/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/**/*.root", recursive=True)
files["ggh"] = glob.glob("/nvmedata/store/mc/RunIIFall17NanoAOD/GluGluHToMuMu_M125_13TeV_amcatnloFXFX_pythia8/NANOAODSIM/**/*.root", recursive=True)

files["data_2017"] = glob.glob("/nvmedata/store/data/Run2017*/SingleMuon/NANOAOD/Nano14Dec2018-v1/**/*.root")

In [ ]:
#remove the friend trees
files["ggh"] = [x for x in files["ggh"] if "Friend" not in x]

In [ ]:
files["ggh"]

In [ ]:
for k, v in files.items():
    print(k, len(v))

In [ ]:
arrays_ev = [
    "PV_npvsGood", "Flag_METFilters", "Flag_goodVertices", "Generator_weight", "genWeight", "HLT_IsoMu24"
]
arrays_jet = [
    "Jet_pt", "Jet_eta", "Jet_phi", "Jet_btagDeepB", "Jet_jetId"
]

arrays_muon = [
    "nMuon", "Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_pfRelIso03_all", "Muon_mediumId", "Muon_charge"
]

arrays_to_load = arrays_jet + arrays_muon

In [ ]:
arrays_gen = [
    "GenPart_pt", "GenPart_eta", "GenPart_phi", "GenPart_mass", "GenPart_pdgId", "GenPart_status"
]

In [ ]:
fn = files["dy"][0]
fi = uproot.open(fn)
tt = fi.get("Events")

In [ ]:
"""
Preloads all data into memory.

Args:
    dataset_files (Dict[str, List[str]]): a dictionary of dataset name to the list of filenames for each dataset

returns:
    Dict[str, List[Dict[str, JaggedArray]]]
    
    For each dataset, a list of awkward-array JaggedArray dictionaries 
"""
def preload_data(dataset_files, arrays_to_load):
    ret = {}
    for dsname, dsfiles in dataset_files.items():
        ret[dsname] = [load_arrays(fn, arrays_to_load) for fn in dsfiles]
    return ret

def load_arrays(fn, arrays_to_load):
    #print("opening file {0}".format(fn))
    fi = uproot.open(fn)
    tt = fi.get("Events")
    with ThreadPoolExecutor(max_workers=16) as executor:
        arrs = tt.arrays(arrays_to_load, executor=executor)
    #print("loaded {0} arrays, shape=({1},)".format(len(arrs), tt.numentries))
    return arrs

In [ ]:
def get_mem():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss/1024.0/1024.0/1024.0

In [ ]:
def print_mem():
    print("Using {0:.2f} GB of RSS".format(get_mem()))

In [ ]:
print_mem()
t0 = time.time()
preloaded_data = preload_data({
    "dy": files["dy"][:2],
     "ggh": files["ggh"][:2],
    "data_2017": files["data_2017"][:2]
    }, arrays_to_load)
print_mem()
t1 = time.time()
dt = t1 - t0

sum_ev = 0
for k in preloaded_data.keys():
    for arrs in preloaded_data[k]:
        sum_ev += len(arrs[b"Jet_pt"])
print("Loaded {0:.2E} events in {1:.0f} seconds, speed {2:.2E} Hz".format(sum_ev, dt, sum_ev/dt))

In [ ]:
pts = preloaded_data["ggh"][0][b"Jet_pt"]
etas = preloaded_data["ggh"][0][b"Jet_eta"]

class Jet:
    def __init__(self, pt, eta):
        self.pt = pt
        self.eta = eta
    def __repr__(self):
        s = "Jet({0:.2f}, {1:.2f})".format(self.pt, self.eta)
        return s

In [ ]:
for iev in range(5):
    jets = []
    for imuon in range(len(pts[iev])):
        jets += [Jet(pts[iev][imuon], etas[iev][imuon])]
    print(len(jets), jets)

In [ ]:
pts.offsets[:6]

In [ ]:
pts.content[:23]

In [ ]:
etas.content[:23]

In [ ]:
ds = hepaccelerate.NanoAODDataset(
    glob.glob("/nvmedata/store/mc/RunIIFall17NanoAOD/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/**/*.root", recursive=True),
    arrays_to_load, "Events", NUMPY_LIB
)

if prepare_cache:
    ds.preload(nthreads=16, do_progress=True, event_vars=[bytes(x, encoding='ascii') for x in arrays_ev])
    ds.to_cache(do_progress=True)
else:
    ds.from_cache(do_progress=True)

ret = ds.analyze(analyze_data_function,
    is_mc=is_mc,
    pu_corrections_target=pu_corrections_2016,
    mu_pt_cut_leading=30, debug=False
)

#ret_35 = ds.analyze(analyze_data, mu_pt_cut_leading=35)
#ret_40 = ds.analyze(analyze_data, mu_pt_cut_leading=40)
ret.save_json("{0}.json".format(datasetname))

In [ ]:
pts = preloaded_data["ggh"][1][b"Jet_pt"]
pts_gpu = cupy.array(pts.content)
bins_cpu = np.linspace(0,500,100)
bins_gpu = cupy.linspace(0,500,100)

In [ ]:
pts.shape[0], pts.content.shape[0]

In [ ]:
%timeit np.histogram(pts.content, bins_cpu)

In [ ]:
%timeit cupy.histogram(pts_cuda, bins_gpu)